In [ ]:
# Importing all necessary package
import os
import numpy as np
from PIL import Image


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

#A generalist algorithm for cell and nucleus segmentation (v1.0) that can be optimized for your own data (v2.0)
from cellpose.models import Cellpose 
from stardist.models import StarDist2D

from csbdeep.utils import normalize

import tensorflow as tf

from tensorflow import keras
from tensorflow.config import list_physical_devices

# Setting the random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Check if GPU is available
if len(list_physical_devices("GPU")) >= 1:
    use_GPU = True
    tf.config.experimental.set_memory_growth(list_physical_devices("GPU")[0], True)

else:
    use_GPU = False

def load_cellpose():
    """Load the cellpose model and return it"""
    model_c = Cellpose(gpu=use_GPU, model_type="cyto2")
    return model_c


def load_stardist():
    """Load the Stardist model and return it"""
    model_s = StarDist2D.from_pretrained("2D_versatile_he")
    return model_s


def run_cellpose(image, model_cellpose, diameter=None):
    """Run the cellpose model on the image and return the mask"""
    channel = [[0, 0]]
    mask_cellpose, _, _, _ = model_cellpose.eval(
        image, batch_size=1, diameter=diameter, channels=channel
    )
    return mask_cellpose


def run_stardist(image, model_stardist, nms_thresh=0.4, prob_thresh=0.5):
    """Run the stardist model on the image and return the mask"""
    img_norm = image / 255
    img_norm = normalize(img_norm, 1, 99.8)
    mask_stardist, _ = model_stardist.predict_instances(
        img_norm, nms_thresh=nms_thresh, prob_thresh=prob_thresh
    )
    return mask_stardist

model_cellpose = load_cellpose()
model_stardist = load_stardist()

In [ ]:
# Load an image, run CellPose to detect muscle fibers and Stardist to detect nuclei.
try:
    from imageio.v2 import imread
except:
    from imageio import imread

img = imread("im") # mon image 

# CellPose
mask_cellpose = run_cellpose(
    img, model_cellpose)
mask_cellpose = mask_cellpose.astype(np.uint16)
cellpose_mask_filename = "results/sample_he_cellpose_mask.tiff"
Image.fromarray(mask_cellpose).save(cellpose_mask_filename)

# Stardist
mask_stardist = run_stardist(
    img, model_stardist)
mask_stardist = mask_stardist.astype(np.uint16)
stardist_filename = "results/sample_he_stardist_mask.tiff"
Image.fromarray(mask_stardist).save(stardist_filename)

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,9))
ax1.imshow(img)
ax2.imshow(mask_cellpose)
ax3.imshow(mask_stardist)
plt.show()

In [ ]:
# Based on the Cellpose Image and Stardist image we will extract the coordinates of the nuclei and muscle fibers as a table. 
# We will also extract their area in pixels, perimeter, eccentrictiy....
from skimage.measure import regionprops_table
import pandas as pd
props_cellpose = regionprops_table(
    mask_cellpose,
    properties=[
        "label",
        "area",
        "centroid",
        "eccentricity",
        "bbox",
        "image",
        "perimeter",
        "feret_diameter_max"#ajout du diamètre
    ],
)
df_cellpose = pd.DataFrame(props_cellpose)
df_cellpose

In [ ]:
df_cellpose.to_csv(r'data_mon im_jpg.csv', index=False, header=True) # je transforme en csv 

In [ ]:
import pandas as pd 
#on importe les fichiers csv nécessaire 
df=pd.read_csv('muscle_atlas_clean.csv') #atlas du muscle
df_cell=pd.read_csv('data_mon im_jpg.csv') # fichier info image 5 (taille, diamètre, centre...)



def conversion_pix_µm(im, data_frame,muscle_atlas):
    'Convertion du diamètre en µm'
    barre_echelle= muscle_atlas.loc[(df['Number']== im), 'Calibration (um/px)']
    
    data_frame['diameter_µm']=(data_frame['feret_diameter_max'])*barre_echelle.values

conversion_pix_µm('mon_im.jpg',df_cell,df )

def count_atrophic_cells (data_frame,min_size, max_size):
    hyper='Hypertrophy'
    hypo= 'Hypotrophy'
    norm= 'Normal'
    data_frame["State_cell"] = ''
    for index in data_frame.index:
        a=data_frame['diameter_µm'][index]
        if a > max_size:
            data_frame["State_cell"][index]=hyper
        elif a < min_size:
            data_frame["State_cell"][index]=hypo
        else: 
            data_frame["State_cell"][index]= norm


count_atrophic_cells (df_cell, 30, 80)

In [ ]:
def pourcentage_state_cell (im,data_frame,muscle_atlas):
     data = {'Number':[],'Hypertrophy': [],'Normal':[],'Hypotrophy': []}  # Ici on créer un dataframe pour mettre ensuite les valeurs des pourcentages
     df_state_cell = pd.DataFrame(data)  
     percentage= (data_frame['State_cell'].value_counts()/data_frame['State_cell'].count())*100
     df_state_cell.loc[len(df_state_cell.index)] = percentage

     
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Hypertrophic_cell_%"]=df_state_cell['Hypertrophy'].values
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Normal_cell_%"]=df_state_cell['Normal'].values
     muscle_atlas.loc[(muscle_atlas['Number']== im),"Hypotrophic_cell_%"]=df_state_cell['Hypotrophy'].values


pourcentage_state_cell ('mon_im.jpg',df_cell,df) 

In [ ]:
def diameter_mean_std (im,data_frame,muscle_atlas):
    cell_mean=data_frame['diameter_µm'].mean()
    cell_std=data_frame['diameter_µm'].std()
    muscle_atlas.loc[(muscle_atlas['Number']== im),"diameter_mean"]=cell_mean
    muscle_atlas.loc[(muscle_atlas['Number']== im),"diameter_std"]=cell_std


diameter_mean_std('mon_im.jpg',df_cell,df)   
df